In [1]:
import torch
from torch.utils.data import Dataset, DataLoader

# 1. Charger le fichier texte
with open('exemple.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    
text = text.strip().lower()

# 2. Vocabulaire
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}

# 3. Encodage
data = torch.tensor([stoi[c] for c in text], dtype=torch.long)
block_size = 64

# 4. Dataset personnalisé
class CharDataset(Dataset):
    def __init__(self, data, block_size):
        self.data = data
        self.block_size = block_size

    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        x = self.data[idx : idx + self.block_size]
        y = self.data[idx + 1 : idx + 1 + self.block_size]
        return x, y

# 5. Créer le DataLoader
batch_size = 4
dataset = CharDataset(data, block_size)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 6. Exemple d’utilisation
x, y = next(iter(dataloader))
print("Texte encodé. Taille vocabulaire :", vocab_size)
print("x shape :", x.shape)  # [batch_size, block_size]
print("y shape :", y.shape)

Texte encodé. Taille vocabulaire : 36
x shape : torch.Size([4, 64])
y shape : torch.Size([4, 64])


In [2]:
import torch.nn as nn
import torch.nn.functional as F

# Définition d’un mini-modèle GPT
class TinyGPT(nn.Module):
    def __init__(self, vocab_size, n_embed=64):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx):
        x = self.embed(idx)
        logits = self.lm_head(x)
        return logits

# Initialisation
model = TinyGPT(vocab_size)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

# Préparer un itérateur persistant sur le dataloader
data_iter = iter(dataloader)

# Entraînement simple
for step in range(500):
    try:
        x, y = next(data_iter)
    except StopIteration:
        data_iter = iter(dataloader)
        x, y = next(data_iter)

    logits = model(x)
    B, T, C = logits.shape
    loss = loss_fn(logits.view(B * T, C), y.view(B * T))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if step % 100 == 0:
        print(f"Étape {step} – Perte : {loss.item():.4f}")

print("✅ Entraînement terminé.")

Étape 0 – Perte : 3.7334
Étape 100 – Perte : 2.5172
Étape 200 – Perte : 2.2097
Étape 300 – Perte : 2.1297
Étape 400 – Perte : 2.0330
✅ Entraînement terminé.


In [3]:
@torch.no_grad()
def generate(model, prompt, max_new_tokens=100, temperature=1.0, top_k=None, top_p=None):
    model.eval()
    idx = torch.tensor([stoi[c] for c in prompt], dtype=torch.long).unsqueeze(0)

    for _ in range(max_new_tokens):
        idx_cond = idx[:, -block_size:]
        logits = model(idx_cond)
        logits = logits[:, -1, :] / temperature  # température

        # Appliquer top-k
        if top_k is not None:
            topk_vals, topk_idx = torch.topk(logits, top_k)
            logits_filtered = torch.full_like(logits, float('-inf'))
            logits_filtered.scatter_(1, topk_idx, topk_vals)
            logits = logits_filtered

        # Appliquer top-p (nucleus)
        if top_p is not None:
            sorted_logits, sorted_indices = torch.sort(logits, descending=True)
            cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

            # Masque les tokens dépassant top_p
            sorted_mask = cumulative_probs > top_p
            sorted_mask[..., 1:] = sorted_mask[..., :-1].clone()
            sorted_mask[..., 0] = 0  # Garder au moins 1

            logits[0, sorted_indices[0][sorted_mask[0]]] = float('-inf')

        probs = F.softmax(logits, dim=-1)
        next_id = torch.multinomial(probs, num_samples=1)
        idx = torch.cat([idx, next_id], dim=1)

    out = ''.join([itos[i.item()] for i in idx[0]])
    return out

In [6]:
prompt = "un matin,"
generated_text = generate(model, prompt, max_new_tokens=200, temperature=0.1, top_k=20, top_p=0.9)

print("🔮 Texte généré :\n")
print(generated_text)

🔮 Texte généré :

un matin, le le mourit le me le le moure le le me le le le le mme le le le le le le le le le le me le moure le le le le le le le le le le le me le le mmit le le le le le le le le le le le mount le mourit le le
